<a href="https://colab.research.google.com/github/jokend1ng/jokend1ng/blob/jokend1ng-ml_LEANING/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import re

from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding,GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
with open('true.txt', 'r', encoding='utf-8') as f:
    texts_true = f.readlines()
    texts_true[0] = texts_true[0].replace('\ufeff', '')

In [4]:
with open('false.txt', 'r', encoding='utf-8') as f:
    texts_false = f.readlines()
    texts_false[0] = texts_false[0].replace('\ufeff', '')

In [5]:
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

100 50 150


In [6]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

[('пусть', 2), ('я', 117), ('приму', 2), ('и', 44), ('себя', 11), ('таким', 3), ('какой', 2), ('есть', 8), ('несовершенен', 1), ('как', 8)]
Пусть я приму и приму себя таким, какой я есть



In [7]:
max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

print( list(tokenizer.word_index.items()) )

[[ 78   1  79 ...  80   1  18]
 [  0   0   0 ...   2   9  81]
 [  0 149 150 ...   2  11  82]
 ...
 [  0   0   0 ...  72   7 536]
 [  0   0   0 ... 537 538 539]
 [  0   0   0 ...  34   3 541]]
[('я', 1), ('и', 2), ('не', 3), ('буду', 4), ('что', 5), ('в', 6), ('меня', 7), ('мне', 8), ('все', 9), ('могу', 10), ('это', 11), ('себе', 12), ('себя', 13), ('то', 14), ('у', 15), ('с', 16), ('за', 17), ('есть', 18), ('как', 19), ('к', 20), ('чтобы', 21), ('жизнь', 22), ('жизни', 23), ('для', 24), ('сегодня', 25), ('когда', 26), ('деньги', 27), ('сделать', 28), ('моих', 29), ('их', 30), ('быть', 31), ('о', 32), ('должен', 33), ('того', 34), ('делаю', 35), ('на', 36), ('время', 37), ('силах', 38), ('мое', 39), ('тело', 40), ('относиться', 41), ('умею', 42), ('моей', 43), ('по', 44), ('доверять', 45), ('никогда', 46), ('бы', 47), ('таким', 48), ('всех', 49), ('часть', 50), ('свободно', 51), ('выражать', 52), ('свои', 53), ('красоту', 54), ('которую', 55), ('свое', 56), ('его', 57), ('котором', 58)

In [8]:
X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indeces = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indeces]
Y = Y[indeces]

(150, 10) (150, 2)


In [9]:
model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length = max_text_len))
model.add(GRU(128, return_sequences=True))
model.add(GRU(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

history = model.fit(X, Y, batch_size=32, epochs=50)

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 128)           128000    
                                                                 
 gru (GRU)                   (None, 10, 128)           99072     
                                                                 
 gru_1 (GRU)                 (None, 64)                37248     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 264,450
Trainable params: 264,450
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
5/5 [==============================] - 6s 25ms/step - loss: 0.6904 - accuracy: 0.6000
Epoch 2/50
5/5 [==============================] - 0s 24ms/step - loss: 0.6836 - accur

In [10]:
def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

In [15]:
t = "Я  отношусь к себе без уважения".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print( sequence_to_text(data[0]) )

['я', 'отношусь', 'к', 'себе', 'без']


In [16]:
res = model.predict(data_pad)
print(res, np.argmax(res), sep='\n')

[[0.9726667  0.02733338]]
0
